In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Parameters
stock = "TSLA"
rolling_period = 252
percentile_value = 90
start_date = "2021-12-01"
end_date = "2024-12-01"

# Data Acquisition:
stock_data = yf.download(stock, start=start_date, end=end_date)
stock_data.reset_index(inplace=True)
stock_data

In [ ]:
# Moving Average Calculation
sma_col = f"MA_{rolling_period}"
stock_data[sma_col] = stock_data["Adj Close"].rolling(window=rolling_period).mean()

# Price Deviation Analysis:
deviation_col = f"Adj_Close_to_{sma_col}"
stock_data[deviation_col] = stock_data["Adj Close"] / stock_data[sma_col]

# Percentile Calculation
percentile_col = f"{percentile_value}th_Percentile"
stock_data[percentile_col] = (
   stock_data[deviation_col]
    .rolling(window=rolling_period)
    .apply(lambda x: np.percentile(x, percentile_value), raw=True)
)
stock_data

In [ ]:
# Visualization

# Chart for Price and SMA
fig = go.Figure()

# Add TSLA Adj Close time series
fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data["Adj Close"],
                         name="Adj Close",
                        mode="lines"))

# Add SMA
fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data[sma_col],
                        name=sma_col,
                        mode="lines"))

fig.update_layout(
     title=f"{stock} Adj Close and SMA",
    xaxis_title="Date",
    yaxis_title="Price",
    template="plotly_white"
)

fig.show()



# Chart for deviation and percentile

fig = go.Figure()

# Add Adj Close/SMA
fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data[deviation_col],
                        name="Price Deviation",
                        mode="lines"))

fig.add_trace(go.Scatter(x=stock_data["Date"], y=stock_data[percentile_col],
                        name=f"{percentile_value}th Percentile",
                        mode="lines"))
fig.update_layout(
     title=f"{stock} Deviation and Percentile",
    xaxis_title="Date",
    template="plotly_white"
)

fig.show()